# Get Congress Members

In [1]:
import requests as rq
from bs4 import BeautifulSoup
import pandas as pd
import os
from itertools import groupby
import numpy as np

In [24]:
url2 = 'https://en.wikipedia.org/wiki/List_of_current_United_States_senators'
response2 = rq.get(url2)

soup2 = BeautifulSoup(response2.content,'html.parser')

tables = soup2.find_all('table')
party_state = pd.read_html(str(tables[5]))
party_state = pd.DataFrame(party_state[0])

split_names = party_state['Senator'].str.split(expand=True)
split_names.columns = ['first_name', 'middle', 'last_name']

party_state = pd.concat([party_state, split_names], axis=1)

In [25]:
party_state

,State,Portrait,Senator,Party,Party.1,Born,Occupation(s),Previous elective office(s),Education,Assumed office,Class,Residence[5],first_name,middle,last_name
0,Alabama,NaN,Tommy Tuberville,NaN,Republican,(age 69),College football coach Investment management f...,NaN,Southern Arkansas University (BS),"January 3, 2021",2026 Class 2,Auburn[6],Tommy,Tuberville,None
1,Alabama,NaN,Katie Britt,NaN,Republican,(age 42),Senate staffer University administrator Lawyer...,NaN,"University of Alabama (BS, JD)","January 3, 2023",2028 Class 3,Montgomery,Katie,Britt,None
2,Alaska,NaN,Lisa Murkowski,NaN,Republican,(age 66),Lawyer,Alaska House of Representatives,Georgetown University (AB) Willamette Universi...,"December 20, 2002[c]",2028 Class 3,Girdwood,Lisa,Murkowski,None
3,Alaska,NaN,Dan Sullivan,NaN,Republican,(age 59),U.S. Marine Corps officer Lawyer Assistant Sec...,Alaska Attorney General,Harvard University (AB) Georgetown University ...,"January 3, 2015",2026 Class 2,Anchorage,Dan,Sullivan,None
4,Arizona,NaN,Kyrsten Sinema,NaN,Independent[d],(age 47),Social worker Political activist Lawyer Colleg...,U.S. House Arizona Senate Arizona House of Rep...,Brigham Young University (BA) Arizona State Un...,"January 3, 2019",2024 Class 1,Phoenix,Kyrsten,Sinema,None
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
95,West Virginia,NaN,Shelley Moore Capito,NaN,Republican,(age 70),College career counselor West Virginia Board o...,U.S. House West Virginia House of Delegates,Duke University (BA) University of Virginia (MEd),"January 3, 2015",2026 Class 2,Charleston,Shelley,Moore,Capito
96,Wisconsin,NaN,Ron Johnson,NaN,Republican,(age 69),Accountant Corporate executive,NaN,University of Minnesota (BS),"January 3, 2011",2028 Class 3,Oshkosh,Ron,Johnson,None
97,Wisconsin,NaN,Tammy Baldwin,NaN,Democratic,(age 62),Lawyer,"U.S. House Wisconsin Assembly Dane County, Wis...",Smith College (BA) University of Wisconsin-Mad...,"January 3, 2013",2024 Class 1,Madison,Tammy,Baldwin,None
98,Wyoming,NaN,John Barrasso,NaN,Republican,(age 71),Orthopedic surgeon Medical chief of staff Nonp...,Wyoming Senate,Rensselaer Polytechnic Institute Georgetown Un...,"June 25, 2007[ad]",2024 Class 1,Casper,John,Barrasso,None


## Get Senate Members

In [3]:
url2 = 'https://www.senate.gov/general/committee_assignments/assignments.htm'
response2 = rq.get(url2)

soup2 = BeautifulSoup(response2.content,'html.parser')

gov_data = soup2.select('a')

actual_titles = []
for title in gov_data:
    if title.parent.name == 'div' or title.parent.name == 'strong':
        actual_titles.append(title.text.strip())


In [4]:
arr = np.array(actual_titles)
idx = np.where(arr == 'Back to top')[0]
subarrays = np.split(arr, idx+1)

result = [subarray.tolist() for subarray in subarrays if len(subarray) > 0]

filtered_result = [[item for item in sublist if item != '' and item != 'Back to top'] for sublist in result]

senators_committees = pd.DataFrame(filtered_result).rename(columns={0:'senator',1:'committee_1',2:'committee_2',3:'committee_3',4:'committee_4',5:'committee_5',6:'committee_6',7:'committee_7',8:'committee_8'})

In [5]:
senators_committees['first_name'] = senators_committees['senator'].str.split(', ').str[1]
senators_committees['last_name'] = senators_committees['senator'].str.split(', ').str[0]
senators_committees['full_name'] = senators_committees['first_name'] + ' ' + senators_committees['last_name']

In [6]:
senators_committees

,senator,committee_1,committee_2,committee_3,committee_4,committee_5,committee_6,committee_7,committee_8,first_name,last_name,full_name
0,"Baldwin, Tammy",Committee on Appropriations,"Committee on Commerce, Science, and Transporta...","Committee on Health, Education, Labor, and Pen...",None,None,None,None,None,Tammy,Baldwin,Tammy Baldwin
1,"Barrasso, John",Committee on Energy and Natural Resources,Committee on Finance,Committee on Foreign Relations,None,None,None,None,None,John,Barrasso,John Barrasso
2,"Bennet, Michael F.","Committee on Agriculture, Nutrition, and Forestry",Committee on Finance,Committee on Rules and Administration,Select Committee on Intelligence,None,None,None,None,Michael F.,Bennet,Michael F. Bennet
3,"Blackburn, Marsha","Committee on Commerce, Science, and Transporta...",Committee on Finance,Committee on the Judiciary,Committee on Veterans' Affairs,None,None,None,None,Marsha,Blackburn,Marsha Blackburn
4,"Blumenthal, Richard",Commission on Security and Cooperation in Europe,Committee on Armed Services,Committee on Homeland Security and Governmenta...,Committee on the Judiciary,Committee on Veterans' Affairs,Special Committee on Aging,United States Senate Caucus on International N...,None,Richard,Blumenthal,Richard Blumenthal
...,...,...,...,...,...,...,...,...,...,...,...,...
96,"Whitehouse, Sheldon",Commission on Security and Cooperation in Europe,Committee on Environment and Public Works,Committee on Finance,Committee on the Budget,Committee on the Judiciary,United States Senate Caucus on International N...,None,None,Sheldon,Whitehouse,Sheldon Whitehouse
97,"Wicker, Roger F.",Commission on Security and Cooperation in Europe,Committee on Armed Services,"Committee on Commerce, Science, and Transporta...",Committee on Environment and Public Works,Committee on Rules and Administration,Select Committee on Intelligence,None,None,Roger F.,Wicker,Roger F. Wicker
98,"Wyden, Ron",Committee on Energy and Natural Resources,Committee on Finance,Committee on the Budget,Joint Committee on Taxation,Select Committee on Intelligence,None,None,None,Ron,Wyden,Ron Wyden
99,"Young, Todd","Committee on Commerce, Science, and Transporta...",Committee on Finance,Committee on Foreign Relations,Committee on Small Business and Entrepreneurship,None,None,None,None,Todd,Young,Todd Young


In [26]:
senators_committees = senators_committees.merge(party_state, left_on='last_name', right_on='Senator', validate='1:1')
senators_committees = senators_committees[['senator', 'first_name', 'last_name', 'full_name', 'committee_1','committee_2','committee_3','committee_4','committee_5','committee_6','committee_7','committee_8','Party.1','State']]

MergeError: Merge keys are not unique in either left or right dataset; not a one-to-one merge

In [8]:
senators_committees

,senator,first_name,last_name,full_name,committee_1,committee_2,committee_3,committee_4,committee_5,committee_6,committee_7,committee_8,Party.1,State
0,"Baldwin, Tammy",Tammy,Baldwin,Tammy Baldwin,Committee on Appropriations,"Committee on Commerce, Science, and Transporta...","Committee on Health, Education, Labor, and Pen...",None,None,None,None,None,Democratic,Wisconsin
1,"Barrasso, John",John,Barrasso,John Barrasso,Committee on Energy and Natural Resources,Committee on Finance,Committee on Foreign Relations,None,None,None,None,None,Republican,Wyoming
2,"Blackburn, Marsha",Marsha,Blackburn,Marsha Blackburn,"Committee on Commerce, Science, and Transporta...",Committee on Finance,Committee on the Judiciary,Committee on Veterans' Affairs,None,None,None,None,Republican,Tennessee
3,"Blumenthal, Richard",Richard,Blumenthal,Richard Blumenthal,Commission on Security and Cooperation in Europe,Committee on Armed Services,Committee on Homeland Security and Governmenta...,Committee on the Judiciary,Committee on Veterans' Affairs,Special Committee on Aging,United States Senate Caucus on International N...,None,Democratic,Connecticut
4,"Boozman, John",John,Boozman,John Boozman,Commission on Security and Cooperation in Europe,"Committee on Agriculture, Nutrition, and Forestry",Committee on Appropriations,Committee on Environment and Public Works,Committee on Veterans' Affairs,None,None,None,Republican,Arkansas
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
68,"Warren, Elizabeth",Elizabeth,Warren,Elizabeth Warren,Committee on Armed Services,"Committee on Banking, Housing, and Urban Affairs",Committee on Finance,Special Committee on Aging,None,None,None,None,Democratic,Massachusetts
69,"Welch, Peter",Peter,Welch,Peter Welch,"Committee on Agriculture, Nutrition, and Forestry","Committee on Commerce, Science, and Transporta...",Committee on Rules and Administration,Committee on the Judiciary,Joint Economic Committee,None,None,None,Democratic,Vermont
70,"Whitehouse, Sheldon",Sheldon,Whitehouse,Sheldon Whitehouse,Commission on Security and Cooperation in Europe,Committee on Environment and Public Works,Committee on Finance,Committee on the Budget,Committee on the Judiciary,United States Senate Caucus on International N...,None,None,Democratic,Rhode Island
71,"Wyden, Ron",Ron,Wyden,Ron Wyden,Committee on Energy and Natural Resources,Committee on Finance,Committee on the Budget,Joint Committee on Taxation,Select Committee on Intelligence,None,None,None,Democratic,Oregon


In [11]:
DATA_PATH = '../../data'

if not os.path.exists(DATA_PATH+'/inputs'):
    os.makedirs(DATA_PATH+'/inputs')

senators_committees_csv = '/inputs/senators_committees.csv'

if not os.path.exists(senators_committees_csv):
    senators_committees.to_csv(DATA_PATH+senators_committees_csv,index=False)

## Get gsectors

In [ ]:
# url3 = 'https://en.wikipedia.org/wiki/Global_Industry_Classification_Standard'
# response3 = rq.get(url3)

# soup3 = BeautifulSoup(response3.content,'html.parser')

# gsect = soup3.select('table')
# gsect = pd.read_html(str(gsect))
# gsect = pd.DataFrame(gsect[0])

In [ ]:
# gsect